# Phân tích dữ liệu CVG (Insurance Dataset)

## Mục tiêu:
1. Vẽ biểu đồ tương quan giữa các biến
2. Histogram của từng biến
3. Hồi quy charges theo các biến còn lại với gradient descent

## Dataset:
- **age** (int): Tuổi
- **sex** (female, male): Giới tính
- **bmi** (double): Chỉ số BMI
- **children** (0, 1): Có con hay không
- **smoker** (yes, no): Hút thuốc hay không
- **region** (string): Khu vực
- **charges** (double): Chi phí bảo hiểm

In [ ]:
# Import các thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
import os
warnings.filterwarnings('ignore')

# Cài đặt style cho plots
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Tạo dữ liệu mẫu (hoặc load từ file CSV)
def create_sample_data(n_samples=1000):
    """
    Tạo dữ liệu mẫu cho demo
    Trong thực tế, bạn có thể load từ file CSV:
    data = pd.read_csv('insurance.csv')
    """
    np.random.seed(42)
    
    # Tạo các biến
    ages = np.random.randint(18, 65, n_samples)
    sexes = np.random.choice(['female', 'male'], n_samples)
    bmis = np.random.normal(26, 4, n_samples)
    bmis = np.clip(bmis, 15, 50)  # Giới hạn BMI hợp lý
    children = np.random.choice([0, 1], n_samples, p=[0.7, 0.3])
    smokers = np.random.choice(['yes', 'no'], n_samples, p=[0.2, 0.8])
    regions = np.random.choice(['northeast', 'northwest', 'southeast', 'southwest'], n_samples)
    
    # Tạo charges có correlation với các biến khác
    charges = (ages * 50 + 
              bmis * 100 + 
              children * 1000 + 
              (sexes == 'male') * 500 +
              (smokers == 'yes') * 5000 + 
              np.random.normal(0, 1000, n_samples))
    charges = np.abs(charges)  # Đảm bảo charges dương
    
    data = pd.DataFrame({
        'age': ages,
        'sex': sexes,
        'bmi': bmis,
        'children': children,
        'smoker': smokers,
        'region': regions,
        'charges': charges
    })
    
    return data

# Tạo dữ liệu
# data = create_sample_data(1000)
# Load data from file
filePath = 'data/insurance.csv'
# Check if file exists
import os
if os.path.exists(filePath):
    data = pd.read_csv(filePath)
    print(f"Loaded data from {filePath}")
else:
    print(f"File {filePath} not found. Creating sample data instead.")
    data = create_sample_data(1000)
    
print("Thông tin dữ liệu:")
print(data.info())
print("\nMẫu dữ liệu:")
print(data.head())
print("\nThống kê mô tả:")
print(data.describe())

## 1. Biểu đồ tương quan giữa các biến

In [ ]:
# Chuẩn bị dữ liệu cho correlation matrix
def prepare_data_for_correlation(data):
    """
    Encode categorical variables để tính correlation
    """
    data_encoded = data.copy()
    
    # Encode categorical variables
    le_sex = LabelEncoder()
    le_smoker = LabelEncoder()
    le_region = LabelEncoder()
    
    data_encoded['sex_encoded'] = le_sex.fit_transform(data['sex'])
    data_encoded['smoker_encoded'] = le_smoker.fit_transform(data['smoker'])
    data_encoded['region_encoded'] = le_region.fit_transform(data['region'])
    
    # Chọn các cột số để tính correlation
    numeric_cols = ['age', 'bmi', 'children', 'sex_encoded', 'smoker_encoded', 'region_encoded', 'charges']
    
    return data_encoded[numeric_cols], le_sex, le_smoker, le_region

data_corr, le_sex, le_smoker, le_region = prepare_data_for_correlation(data)

# Tính correlation matrix
correlation_matrix = data_corr.corr()

# Vẽ correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='coolwarm', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={"shrink": .8})
plt.title('Ma trận tương quan giữa các biến', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("Correlation với charges:")
charges_corr = correlation_matrix['charges'].sort_values(ascending=False)
for var, corr in charges_corr.items():
    if var != 'charges':
        print(f"{var}: {corr:.3f}")

In [ ]:
# Scatter plots giữa các biến số và charges
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Biểu đồ tương quan giữa các biến và Charges', fontsize=16, fontweight='bold')

# Age vs Charges
axes[0,0].scatter(data['age'], data['charges'], alpha=0.6, color='blue')
axes[0,0].set_xlabel('Age')
axes[0,0].set_ylabel('Charges')
axes[0,0].set_title('Age vs Charges')

# BMI vs Charges
axes[0,1].scatter(data['bmi'], data['charges'], alpha=0.6, color='green')
axes[0,1].set_xlabel('BMI')
axes[0,1].set_ylabel('Charges')
axes[0,1].set_title('BMI vs Charges')

# Children vs Charges (boxplot)
sns.boxplot(data=data, x='children', y='charges', ax=axes[0,2])
axes[0,2].set_title('Children vs Charges')

# Sex vs Charges (boxplot)
sns.boxplot(data=data, x='sex', y='charges', ax=axes[1,0])
axes[1,0].set_title('Sex vs Charges')

# Smoker vs Charges (boxplot)
sns.boxplot(data=data, x='smoker', y='charges', ax=axes[1,1])
axes[1,1].set_title('Smoker vs Charges')

# Region vs Charges (boxplot)
sns.boxplot(data=data, x='region', y='charges', ax=axes[1,2])
axes[1,2].tick_params(axis='x', rotation=45)
axes[1,2].set_title('Region vs Charges')

plt.tight_layout()
plt.show()

## 2. Histogram của từng biến

In [ ]:
# Vẽ histogram cho các biến số
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Histogram của các biến số', fontsize=16, fontweight='bold')

# Age
axes[0,0].hist(data['age'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_xlabel('Age')
axes[0,0].set_ylabel('Frequency')
axes[0,0].set_title(f'Age Distribution (Mean: {data["age"].mean():.1f})')
axes[0,0].axvline(data['age'].mean(), color='red', linestyle='--', label='Mean')
axes[0,0].legend()

# BMI
axes[0,1].hist(data['bmi'], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0,1].set_xlabel('BMI')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title(f'BMI Distribution (Mean: {data["bmi"].mean():.1f})')
axes[0,1].axvline(data['bmi'].mean(), color='red', linestyle='--', label='Mean')
axes[0,1].legend()

# Children
children_counts = data['children'].value_counts().sort_index()
axes[1,0].bar(children_counts.index, children_counts.values, alpha=0.7, color='orange', edgecolor='black')
axes[1,0].set_xlabel('Children')
axes[1,0].set_ylabel('Frequency')
axes[1,0].set_title('Children Distribution')
axes[1,0].set_xticks([0, 1])

# Charges
axes[1,1].hist(data['charges'], bins=30, alpha=0.7, color='coral', edgecolor='black')
axes[1,1].set_xlabel('Charges')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title(f'Charges Distribution (Mean: {data["charges"].mean():.0f})')
axes[1,1].axvline(data['charges'].mean(), color='red', linestyle='--', label='Mean')
axes[1,1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Vẽ bar charts cho các biến categorical
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Phân phối các biến categorical', fontsize=16, fontweight='bold')

# Sex
sex_counts = data['sex'].value_counts()
axes[0].bar(sex_counts.index, sex_counts.values, alpha=0.7, color=['pink', 'lightblue'], edgecolor='black')
axes[0].set_xlabel('Sex')
axes[0].set_ylabel('Count')
axes[0].set_title('Sex Distribution')
for i, v in enumerate(sex_counts.values):
    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Smoker
smoker_counts = data['smoker'].value_counts()
axes[1].bar(smoker_counts.index, smoker_counts.values, alpha=0.7, color=['lightgreen', 'lightcoral'], edgecolor='black')
axes[1].set_xlabel('Smoker')
axes[1].set_ylabel('Count')
axes[1].set_title('Smoker Distribution')
for i, v in enumerate(smoker_counts.values):
    axes[1].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Region
region_counts = data['region'].value_counts()
axes[2].bar(range(len(region_counts)), region_counts.values, alpha=0.7, color='gold', edgecolor='black')
axes[2].set_xlabel('Region')
axes[2].set_ylabel('Count')
axes[2].set_title('Region Distribution')
axes[2].set_xticks(range(len(region_counts)))
axes[2].set_xticklabels(region_counts.index, rotation=45)
for i, v in enumerate(region_counts.values):
    axes[2].text(i, v + 5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 3. Hồi quy Charges với Gradient Descent

In [ ]:
class GradientDescentRegression:
    def __init__(self, learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        self.weights = None
        self.bias = None
        self.cost_history = []
        
    def fit(self, X, y):
        """
        Huấn luyện mô hình với gradient descent
        """
        # Khởi tạo weights và bias
        n_features = X.shape[1]
        self.weights = np.random.normal(0, 0.01, n_features)
        self.bias = 0
        
        # Gradient descent
        prev_cost = float('inf')
        
        for i in range(self.max_iterations):
            # Forward pass
            y_pred = self.predict(X)
            
            # Tính cost (MSE)
            cost = np.mean((y_pred - y) ** 2)
            self.cost_history.append(cost)
            
            # Tính gradients
            n_samples = X.shape[0]
            dw = (2/n_samples) * np.dot(X.T, (y_pred - y))
            db = (2/n_samples) * np.sum(y_pred - y)
            
            # Update parameters
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
            
            # Check convergence
            if abs(prev_cost - cost) < self.tolerance:
                print(f"Converged at iteration {i+1}")
                break
            prev_cost = cost
            
            if (i+1) % 100 == 0:
                print(f"Iteration {i+1}, Cost: {cost:.2f}")
    
    def predict(self, X):
        """
        Dự đoán
        """
        return np.dot(X, self.weights) + self.bias
    
    def get_cost_history(self):
        return self.cost_history

In [ ]:
# Chuẩn bị dữ liệu cho regression
def prepare_data_for_regression(data):
    """
    Chuẩn bị dữ liệu cho regression
    """
    # Tạo bản copy
    data_reg = data.copy()
    
    # Encode categorical variables
    le_sex = LabelEncoder()
    le_smoker = LabelEncoder()
    le_region = LabelEncoder()
    
    data_reg['sex_encoded'] = le_sex.fit_transform(data['sex'])
    data_reg['smoker_encoded'] = le_smoker.fit_transform(data['smoker'])
    data_reg['region_encoded'] = le_region.fit_transform(data['region'])
    
    # Chọn features
    feature_cols = ['age', 'bmi', 'children', 'sex_encoded', 'smoker_encoded', 'region_encoded']
    X = data_reg[feature_cols].values
    y = data_reg['charges'].values
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, y, feature_cols, scaler, le_sex, le_smoker, le_region

# Chuẩn bị dữ liệu
X, y, feature_names, scaler, le_sex, le_smoker, le_region = prepare_data_for_regression(data)

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Number of features: {X_train.shape[1]}")
print(f"Feature names: {feature_names}")

In [ ]:
# Huấn luyện mô hình Gradient Descent
print("Đang huấn luyện mô hình Gradient Descent...")
gd_model = GradientDescentRegression(learning_rate=0.01, max_iterations=1000)
gd_model.fit(X_train, y_train)

# Dự đoán
y_train_pred = gd_model.predict(X_train)
y_test_pred = gd_model.predict(X_test)

# Đánh giá mô hình
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("\n=== KẾT QUẢ MÔ HÌNH ===")
print(f"Training MSE: {train_mse:.2f}")
print(f"Test MSE: {test_mse:.2f}")
print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

print("\n=== TRỌNG SỐ CỦA CÁC BIẾN ===")
for i, (feature, weight) in enumerate(zip(feature_names, gd_model.weights)):
    print(f"{feature}: {weight:.4f}")
print(f"Bias: {gd_model.bias:.4f}")

In [ ]:
# Visualize kết quả
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Kết quả Gradient Descent Regression', fontsize=16, fontweight='bold')

# 1. Cost function history
axes[0,0].plot(gd_model.get_cost_history(), color='blue', linewidth=2)
axes[0,0].set_xlabel('Iteration')
axes[0,0].set_ylabel('Cost (MSE)')
axes[0,0].set_title('Cost Function History')
axes[0,0].grid(True, alpha=0.3)

# 2. Actual vs Predicted (Training)
axes[0,1].scatter(y_train, y_train_pred, alpha=0.6, color='blue', label='Training')
min_val = min(y_train.min(), y_train_pred.min())
max_val = max(y_train.max(), y_train_pred.max())
axes[0,1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
axes[0,1].set_xlabel('Actual Charges')
axes[0,1].set_ylabel('Predicted Charges')
axes[0,1].set_title(f'Training Set (R² = {train_r2:.4f})')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Actual vs Predicted (Test)
axes[1,0].scatter(y_test, y_test_pred, alpha=0.6, color='green', label='Test')
min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
axes[1,0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
axes[1,0].set_xlabel('Actual Charges')
axes[1,0].set_ylabel('Predicted Charges')
axes[1,0].set_title(f'Test Set (R² = {test_r2:.4f})')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. Feature Importance (Weights)
weights_abs = np.abs(gd_model.weights)
sorted_idx = np.argsort(weights_abs)[::-1]
sorted_features = [feature_names[i] for i in sorted_idx]
sorted_weights = weights_abs[sorted_idx]

bars = axes[1,1].bar(range(len(sorted_features)), sorted_weights, color='orange', alpha=0.7, edgecolor='black')
axes[1,1].set_xlabel('Features')
axes[1,1].set_ylabel('Absolute Weight')
axes[1,1].set_title('Feature Importance (Absolute Weights)')
axes[1,1].set_xticks(range(len(sorted_features)))
axes[1,1].set_xticklabels(sorted_features, rotation=45)

# Thêm giá trị lên bars
for bar, weight in zip(bars, sorted_weights):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'{weight:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Phân tích Residuals', fontsize=16, fontweight='bold')

# 1. Residuals vs Predicted
residuals_train = y_train - y_train_pred
residuals_test = y_test - y_test_pred

axes[0].scatter(y_train_pred, residuals_train, alpha=0.6, color='blue', label='Training')
axes[0].scatter(y_test_pred, residuals_test, alpha=0.6, color='green', label='Test')
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Charges')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residuals vs Predicted Values')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Histogram of residuals
all_residuals = np.concatenate([residuals_train, residuals_test])
axes[1].hist(all_residuals, bins=30, alpha=0.7, color='purple', edgecolor='black')
axes[1].axvline(np.mean(all_residuals), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(all_residuals):.2f}')
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Residuals')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean of residuals: {np.mean(all_residuals):.4f}")
print(f"Std of residuals: {np.std(all_residuals):.4f}")

In [ ]:
# So sánh với Linear Regression từ sklearn
from sklearn.linear_model import LinearRegression

# Huấn luyện sklearn LinearRegression
sklearn_model = LinearRegression()
sklearn_model.fit(X_train, y_train)

# Dự đoán
sklearn_train_pred = sklearn_model.predict(X_train)
sklearn_test_pred = sklearn_model.predict(X_test)

# Đánh giá
sklearn_train_r2 = r2_score(y_train, sklearn_train_pred)
sklearn_test_r2 = r2_score(y_test, sklearn_test_pred)
sklearn_train_mse = mean_squared_error(y_train, sklearn_train_pred)
sklearn_test_mse = mean_squared_error(y_test, sklearn_test_pred)

print("=== SO SÁNH KẾT QUẢ ===")
print("\nGradient Descent (Custom):")
print(f"  Training R²: {train_r2:.4f}, MSE: {train_mse:.2f}")
print(f"  Test R²: {test_r2:.4f}, MSE: {test_mse:.2f}")

print("\nLinear Regression (Sklearn):")
print(f"  Training R²: {sklearn_train_r2:.4f}, MSE: {sklearn_train_mse:.2f}")
print(f"  Test R²: {sklearn_test_r2:.4f}, MSE: {sklearn_test_mse:.2f}")

print("\n=== SO SÁNH WEIGHTS ===")
print("Feature\t\t\tGradient Descent\tSklearn")
print("-" * 50)
for i, feature in enumerate(feature_names):
    print(f"{feature:<15}\t{gd_model.weights[i]:>10.4f}\t\t{sklearn_model.coef_[i]:>10.4f}")
print(f"{'Bias':<15}\t{gd_model.bias:>10.4f}\t\t{sklearn_model.intercept_:>10.4f}")

## Tổng kết

### 1. Phân tích tương quan:
- Các biến có tương quan mạnh nhất với charges
- Mối quan hệ giữa các biến độc lập

### 2. Phân phối dữ liệu:
- Histogram cho thấy phân phối của từng biến
- Giúp hiểu đặc điểm của dataset

### 3. Mô hình Gradient Descent:
- Implementation from scratch của gradient descent
- So sánh với sklearn LinearRegression
- Phân tích residuals và feature importance

### 4. Kết quả:
- R² score cho thấy khả năng giải thích của mô hình
- MSE đo lường độ chính xác dự đoán
- Feature weights cho biết tầm quan trọng của từng biến